In [8]:
# Cache with state dependent routing
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [9]:
# First demonstrate invalid routing setup that will trigger an error
print('First demonstrating invalid routing setup for a cache that triggers an exception:\n')

model = Network('model')

# Model parameters
n = 5  # number of items
m = 2  # cache capacity

# Create nodes (without Router initially to demonstrate error)
source = Source(model, 'Source')
cacheNode = Cache(model, 'Cache', n, m, ReplacementStrategy.FIFO)
delay1 = Delay(model, 'Delay1')
delay2 = Delay(model, 'Delay2')
sink = Sink(model, 'Sink')

# Create classes
jobClass = OpenClass(model, 'InitClass', 0)
hitClass = OpenClass(model, 'HitClass', 0)
missClass = OpenClass(model, 'MissClass', 0)

# Set arrival process
source.set_arrival(jobClass, Exp(2))

# Set service processes
delay1.set_service(hitClass, Exp(10))
delay1.set_service(missClass, Exp(1))
delay2.set_service(hitClass, Exp(20))
delay2.set_service(missClass, Exp(2))

# Set cache configuration
# Uniform item references - use Python list instead of Matrix
prob_array = [1.0/n] * n
pAccess = DiscreteSampler(prob_array)
cacheNode.set_read(jobClass, pAccess)
cacheNode.set_hit_class(jobClass, hitClass)
cacheNode.set_miss_class(jobClass, missClass)

# Add links (direct from cache to delays - this will cause error)
model.add_link(source, cacheNode)
model.add_link(cacheNode, delay1)
model.add_link(cacheNode, delay2)
model.add_link(delay1, sink)
model.add_link(delay2, sink)

# Set routing
source.set_prob_routing(jobClass, cacheNode, 1.0)

# Try to set routing strategy on cache node - this should trigger an error
try:
    cacheNode.set_routing(hitClass, RoutingStrategy.RROBIN)
    print('ERROR: Setting routing strategy on cache node should have failed!')
except Exception as e:
    print(f'Expected error occurred: {e}')
    print('\nWe now illustrate the correct setup of the same model using a Router node.\n')

First demonstrating invalid routing setup for a cache that triggers an exception:

ERROR: Setting routing strategy on cache node should have failed!


## Correct Model Setup with Router Node

In [10]:
# Create correct model with Router node
model = Network('model')

# Model parameters
n = 5  # number of items
m = 2  # cache capacity

# Create nodes (now including Router)
source = Source(model, 'Source')
cacheNode = Cache(model, 'Cache', n, m, ReplacementStrategy.FIFO)
routerNode = Router(model, 'Router')  # Router node for correct routing
delay1 = Delay(model, 'Delay1')
delay2 = Delay(model, 'Delay2')
sink = Sink(model, 'Sink')

# Create classes
jobClass = OpenClass(model, 'InitClass', 0)
hitClass = OpenClass(model, 'HitClass', 0)
missClass = OpenClass(model, 'MissClass', 0)

# Set arrival process
source.set_arrival(jobClass, Exp(2))

# Set service processes
delay1.set_service(hitClass, Exp(10))
delay1.set_service(missClass, Exp(1))
delay2.set_service(hitClass, Exp(20))
delay2.set_service(missClass, Exp(2))

# Set cache configuration
# Uniform item references - use Python list instead of Matrix
prob_array = [1.0/n] * n
pAccess = DiscreteSampler(prob_array)
cacheNode.set_read(jobClass, pAccess)
cacheNode.set_hit_class(jobClass, hitClass)
cacheNode.set_miss_class(jobClass, missClass)

# Add links (now using Router as intermediary)
model.add_link(source, cacheNode)
model.add_link(cacheNode, routerNode)  # Cache to Router
model.add_link(routerNode, delay1)     # Router to Delay1
model.add_link(routerNode, delay2)     # Router to Delay2
model.add_link(delay1, sink)
model.add_link(delay2, sink)

# Set routing
source.set_prob_routing(jobClass, cacheNode, 1.0)

# Cache routes both hit and miss to router
cacheNode.set_prob_routing(hitClass, routerNode, 1.0)
cacheNode.set_prob_routing(missClass, routerNode, 1.0)

# Router uses RAND strategy for both classes
routerNode.set_routing(hitClass, RoutingStrategy.RAND)
routerNode.set_routing(missClass, RoutingStrategy.RAND)

# Delays route to sink
delay1.set_prob_routing(hitClass, sink, 1.0)
delay1.set_prob_routing(missClass, sink, 1.0)
delay2.set_prob_routing(hitClass, sink, 1.0)
delay2.set_prob_routing(missClass, sink, 1.0)

## Solve with Different Solvers

In [11]:
# Initialize solver list
solvers = []

# CTMC - may fail on some cache models with singular matrices
solvers.append(CTMC(model, keep=False, cutoff=1, seed=1))
print('SOLVER: CTMC')
try:
    AvgTable1 = solvers[0].avg_node_table()
    print(AvgTable1)
except Exception as e:
    print(f'CTMC solver failed (known limitation with some cache models): {type(e).__name__}')

SOLVER: CTMC


      Node   JobClass    QLen    Util  RespT  ResidT        ArvR        Tput
0   Source  InitClass  0.0000  0.0000   0.00    0.00  0.0000e+00  6.3194e-01
3    Cache  InitClass  0.0000  0.0000   0.00    0.00  6.3194e-01  0.0000e+00
4    Cache   HitClass  0.0000  0.0000   0.00    0.00  0.0000e+00  2.5278e-01
5    Cache  MissClass  0.0000  0.0000   0.00    0.00  0.0000e+00  3.7917e-01
7   Router   HitClass  0.0000  0.0000   0.00    0.00  2.5278e-01  2.5278e-01
8   Router  MissClass  0.0000  0.0000   0.00    0.00  3.7917e-01  3.7917e-01
10  Delay1   HitClass  0.2305  0.2305   0.10    0.04  1.2639e-01  1.2639e-01
11  Delay1  MissClass  0.3006  0.3006   1.00    0.60  1.8958e-01  1.8958e-01
12  Delay2  InitClass  0.0000  0.0000   0.00    0.00  3.0068e-17  3.0068e-17
13  Delay2   HitClass  0.2275  0.2275   0.05    0.02  1.2639e-01  1.2639e-01
14  Delay2  MissClass  0.2797  0.2797   0.50    0.30  1.8958e-01  1.8958e-01
15    Sink  InitClass  0.0000  0.0000   0.00    0.00  3.0068e-17  0.0000e+00

In [5]:
# Reset model
model.reset()

# SSA
solvers.append(SSA(model, samples=10000, verbose=True, method='serial', seed=23000))
print('\nSOLVER: SSA')
AvgTable2 = solvers[1].avg_node_table()


SOLVER: SSA
SSA samples:   10000
      Node   JobClass    QLen    Util  RespT  ResidT        ArvR        Tput
0   Source  InitClass  0.0000  0.0000   0.00  0.0000  0.0000e+00  2.0000e+00
3    Cache  InitClass  0.0000  0.0000   0.00  0.0000  2.0000e+00  0.0000e+00
4    Cache   HitClass  0.0000  0.0000   0.00  0.0000  0.0000e+00  8.3230e-01
5    Cache  MissClass  0.0000  0.0000   0.00  0.0000  0.0000e+00  1.1677e+00
7   Router   HitClass  0.0000  0.0000   0.00  0.0000  8.3230e-01  8.3230e-01
8   Router  MissClass  0.0000  0.0000   0.00  0.0000  1.1677e+00  1.1677e+00
10  Delay1   HitClass  0.0367  0.0367   0.10  0.0416  4.1615e-01  4.1615e-01
11  Delay1  MissClass  0.6096  0.6096   1.00  0.5838  5.8385e-01  5.8385e-01
12  Delay2  InitClass  0.0000  0.0000   0.00  0.0000  3.8065e-16  3.8065e-16
13  Delay2   HitClass  0.0204  0.0204   0.05  0.0208  4.1615e-01  4.1615e-01
14  Delay2  MissClass  0.3089  0.3089   0.50  0.2919  5.8385e-01  5.8385e-01
15    Sink  InitClass  0.0000  0.0000   0.

In [6]:
# Reset model
model.reset()

# MVA
solvers.append(MVA(model))
print('\nSOLVER: MVA')
AvgTable3 = solvers[2].avg_node_table()


SOLVER: MVA
      Node   JobClass  QLen  Util  RespT  ResidT        ArvR        Tput
0   Source  InitClass  0.00  0.00   0.00    0.00  0.0000e+00  2.0000e+00
3    Cache  InitClass  0.00  0.00   0.00    0.00  2.0000e+00  0.0000e+00
4    Cache   HitClass  0.00  0.00   0.00    0.00  0.0000e+00  8.0000e-01
5    Cache  MissClass  0.00  0.00   0.00    0.00  0.0000e+00  1.2000e+00
7   Router   HitClass  0.00  0.00   0.00    0.00  8.0000e-01  8.0000e-01
8   Router  MissClass  0.00  0.00   0.00    0.00  1.2000e+00  1.2000e+00
10  Delay1   HitClass  0.04  0.04   0.10    0.04  4.0000e-01  4.0000e-01
11  Delay1  MissClass  0.60  0.60   1.00    0.60  6.0000e-01  6.0000e-01
12  Delay2  InitClass  0.00  0.00   0.00    0.00  1.9032e-16  1.9032e-16
13  Delay2   HitClass  0.02  0.02   0.05    0.02  4.0000e-01  4.0000e-01
14  Delay2  MissClass  0.30  0.30   0.50    0.30  6.0000e-01  6.0000e-01
15    Sink  InitClass  0.00  0.00   0.00    0.00  1.9032e-16  0.0000e+00
16    Sink   HitClass  0.00  0.00   0.

In [7]:
# Reset model
model.reset()

# NC
solvers.append(NC(model))
print('\nSOLVER: NC')
AvgTable4 = solvers[3].avg_node_table()
print(AvgTable4)

# Get cache metrics
print(f'\nHit Ratio: {cacheNode.hit_ratio()}')
print(f'Miss Ratio: {cacheNode.miss_ratio()}')


SOLVER: NC
      Node   JobClass    QLen    Util  RespT  ResidT    ArvR    Tput
0   Source  InitClass  0.0000  0.0000   0.00  0.0000  0.0000  2.0000
3    Cache  InitClass  0.0000  0.0000   0.00  0.0000  2.0000  0.0000
4    Cache   HitClass  0.0000  0.0000   0.00  0.0000  0.0000  0.7885
5    Cache  MissClass  0.0000  0.0000   0.00  0.0000  0.0000  1.2115
7   Router   HitClass  0.0000  0.0000   0.00  0.0000  0.7885  0.7885
8   Router  MissClass  0.0000  0.0000   0.00  0.0000  1.2115  1.2115
10  Delay1   HitClass  0.0394  0.0394   0.10  0.0394  0.3943  0.3943
11  Delay1  MissClass  0.6057  0.6057   1.00  0.6057  0.6057  0.6057
13  Delay2   HitClass  0.0197  0.0197   0.05  0.0197  0.3943  0.3943
14  Delay2  MissClass  0.3029  0.3029   0.50  0.3029  0.6057  0.6057
16    Sink   HitClass  0.0000  0.0000   0.00  0.0000  0.7885  0.0000
17    Sink  MissClass  0.0000  0.0000   0.00  0.0000  1.2115  0.0000
      Node   JobClass    QLen    Util  RespT  ResidT    ArvR    Tput
0   Source  InitClass 